### 

## Soft Mixture of Experts

In [2]:
import jax
import jax.numpy as jnp
import flax.nnx as nnx

In [ ]:
class SoftMoE(nnx.Module):
    def __init__(self, n_experts, capacity, n_dim, rngs):
        init = nnx.initializers.normal(stddev=0.02)
        self.gate = nnx.Param(
            init(rngs.default(), (n_experts, n_dim, capacity))
        )
        self.experts = nnx.Param(
            init(rngs.default(), (n_experts, n_dim, n_dim))
        )
    
    def _dispatch(self, x):
    
    def __call__(self, x):
        B, T, C = x.shape
        x = x.reshape(-1, C)
        gate_output = jnp.einsum('bc,ecm->bem', x, self.gate) # B, E, capacity

        expert_inputs = jnp.einsum('')

        

        return gate_probs

In [19]:
B, T, C = 16, 24, 3
n_experts = 8
capacity = B * T // n_experts
print(capacity)
rngs = nnx.Rngs(default=0)
m = SoftMoE(n_experts, capacity, C, rngs)
x = jax.random.normal(rngs.default(), (B, T, C))
y = m(x)
y.shape


48


(8, 48, 384)